# Utils

> Exporting to python module

In [ ]:
#| default_exp core.utils

In [ ]:
#|export
import logging
import os
import nbdev

In [ ]:
#|export
def set_log_level (logger, log_level):
    logger.setLevel(log_level)
    ch = logging.StreamHandler()
    ch.setLevel(log_level)
    logger.addHandler(ch)
    
def get_repo_root_folder (
    file_to_look_for_in_root_folder='settings.ini',
    max_parent_levels_to_traverse=10,
):
    """Gets root folder of repo where notebook is.
    
    It assumes that the root folder has a file called `file_to_look_for_in_root_folder`, which 
    by default is `settings.ini`.
    """
    list_contents = os.listdir ('.')
    traversed_parent_levels=0
    while file_to_look_for_in_root_folder not in list_contents and traversed_parent_levels < max_parent_levels_to_traverse:
        traversed_parent_levels += 1
        os.chdir ('..')
        list_contents = os.listdir ('.')
    repo_root_folder = Path('.').resolve()
    return repo_root_folder

def _cd_root_nbdev_impl ():
    config=nbdev.config.get_config()
    os.chdir(config.config_path)
    
def cd_root ():
    repo_root_path = get_repo_root_folder ()
    os.chdir(repo_root_path)